In [2]:
import pandas as pd
import numpy as np
import json
from math import radians, sin, cos, acos
from tqdm import tqdm
import requests
from itertools import zip_longest
import nltk
import re
import os
from operator import attrgetter

from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlite3 as sql
import psycopg2

from datetime import datetime
import time

from sklearn import preprocessing, feature_extraction, metrics, tree
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, SGDClassifier, LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline, FeatureUnion, make_union
from sklearn.utils import class_weight
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support, classification_report, recall_score
from sklearn.metrics import r2_score, balanced_accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from collections import Counter
from sklearn.tree import DecisionTreeClassifier, export_graphviz
import eli5
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn.kernel_approximation import RBFSampler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

from IPython.display import Audio, display

from keras.models import Sequential
from keras import backend as K
from keras import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam, SGD
from keras.layers import Dense, BatchNormalization
from keras.callbacks import LearningRateScheduler, Callback

import lookahead
import tensorflow as tf
import tensorflow_hub as hub

import gensim, logging
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

postgres_str = ("...")
cnx = create_engine(postgres_str)
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic



## Which invited candidates applied?

In [3]:
def import_data(cnx, shortlist=False):  # returns columns job_profile_d and whether applied and shortlisted
    Status_trans = pd.read_sql_query('''SELECT * FROM ops_StatusTransition;''', cnx)
    Stat = Status_trans[Status_trans['to_status'] == 'invited-apply']['job_profile_id']
    Status_trans = Status_trans[Status_trans['job_profile_id'].isin(Stat.tolist())]
    invited_ = Status_trans[Status_trans['to_status'] == 'invited-apply'].drop_duplicates(subset='job_profile_id')
    applied_ = Status_trans[Status_trans['to_status'] == 'applied'].drop_duplicates(subset='job_profile_id')
    invited_['applied'] = invited_['job_profile_id'].isin(applied_['job_profile_id']).astype(int)
    if shortlist is True:
        shortlisted_ = Status_trans[Status_trans['to_status'] == 'added-to-shortlist']
        shortlisted_.drop_duplicates(subset='job_profile_id', inplace=True)
        invited_['shortlisted'] = invited_['job_profile_id'].isin(shortlisted_['job_profile_id']).astype(int)
        return invited_[['job_profile_id', 'created', 'applied', 'shortlisted']]
    return invited_[['job_profile_id', 'created', 'applied']]

## How was a candidate sourced

In [ ]:
def split_source(value):  # Splits sources
    sources = value['source'].split(" | ")
    value['source2'] = sources[1]
    value['source'] = sources[0]
    return(value)


def internal_s(value):  # Numbers internal sourced candidates
    if value.source == 'Jump Search':
        return 1
    elif value.source == 'ShortList':
        return 2
    elif value.source == 'Talent Pool':
        return 3
    else:
        return 0


def external_s(value):  # Numbers external sourced candidates
    if value.source == 'Reed Search' or value.source == 'Reed':
        return 1
    elif value.source == 'SecsintheCity':
        return 2
    elif value.source == 'Totally Legal':
        return 3
    elif value.source == 'Jump Direct':
        return 4
    elif value.source == 'GAAPWeb':
        return 5
    elif value.source == 'TotalJobs':
        return 6
    elif value.source == 'Jobsite':
        return 7
    elif value.source == 'External Job Post':
        return 8
    elif value.source == 'Outreach':
        return 8
    elif value.source == 'Indeed':
        return 9
    elif value.source == 'SimplyJobs':
        return 10
    elif value.source == 'Scraper':
        if value.source2 == 'Sitc':
            return 2
        elif value.source2 == 'Totallylegal':
            return 3
        else:
            return 5
    elif value.source == 'Upload CV':
        if value.source2 == 'Totaljobs':
            return 6
        elif value.source2 == 'Totally_Legal':
            return 3
        elif value.source2 == 'Jobsite':
            return 7
        elif value.source2 == 'Indeed':
            return 9
        elif value.source2 == 'Gaapweb':
            return 5
        elif value.source2 == 'Secsinthecity':
            return 2
        elif value.source2 == 'Cvlibrary':
            return 11
        elif value.source2 == 'Simplyjobs':
            return 10
        elif value.source2 == 'Reed':
            return 1
        else:
            return 0
    elif value.source == 'Custom':
        if value.source2 == 'Indeed Messaging Tracking Link' or value.source2 == 'indeed':
            return 10
        elif value.source2 == 'TotalJobs':
            return 6
        else:
            return 8
    elif value.source == 'Scraper':
        if value.source2 == 'Sitc':
            return 2
        elif value.source2 == 'Totallylegal':
            return 3
        else:
            return 5
    else:
        return 0


def get_sourcing():
    with open('sourc_0.json') as f:
        data0 = json.load(f)
    with open('new_apps.json') as f:
        data05 = json.load(f)
    with open('sourc_1.json') as f:
        data1 = json.load(f)
    with open('sourc_2.json') as f:
        data2 = json.load(f)
    with open('sourc_3.json') as f:
        data3 = json.load(f)
    with open('sourc_4.json') as f:
        data4 = json.load(f)
    with open('sourc_5.json') as f:
        data5 = json.load(f)
    with open('sourc_6.json') as f:
        data6 = json.load(f)
    with open('sourc_7.json') as f:
        data7 = json.load(f)
    with open('sourc__0.json') as f:
        data8 = json.load(f)
    with open('sourc__1.json') as f:
        data9 = json.load(f)
    with open('sourc__2.json') as f:
        data10 = json.load(f)
    with open('sourc__3.json') as f:
        data11 = json.load(f)
    with open('sourc__4.json') as f:
        data12 = json.load(f)
    with open('sourc__5.json') as f:
        data13 = json.load(f)
    z = {**data0, **data05, **data1, **data2, **data3, **data4, **data5, **data6,
         **data7, **data8, **data9, **data10, **data11, **data12, **data13}
    data_ = pd.Series(z).to_frame()
    data_.columns = ['source']
    data_ = data_.dropna(subset=['source'])
    data_['job_profile_id'] = data_.index
    data_ = data_.set_index(pd.Series(range(len(data_))))
    data_['source2'] = ""
    temp_var = data_[data_['source'].str.contains(' | ', regex=False)].apply(split_source, axis=1)
    data_[data_['source'].str.contains(' | ', regex=False)] = temp_var
    data_['s_external_overall'] = data_['s_internal_overall'] = 0
    data_ = data_.assign(s_internal_overall=data_.apply(internal_s, axis=1))
    data_ = data_.assign(s_external_overall=data_.apply(external_s, axis=1))
    data_ = pd.get_dummies(data_, prefix=['source_i'], columns=['s_internal_overall'])
    data_ = pd.get_dummies(data_, prefix=['source_e'], columns=['s_external_overall'])
    return data_

In [ ]:
def compute_application_rate(data):
    return data['applied'].mean() * 100


def compute_application_rate_per(data, company=True, job=False):
    merged_data = pd.merge(data, JobProfile, how="inner", left_on="job_profile_id",
                           right_on="job_profile_id", suffixes=("_", "_Job"))
    merged_data = pd.merge(merged_data, Job_Post, how="inner", left_on="job_post_id",
                           right_on="job_post_id", suffixes=("_", "_Job"))
    if job is True:
        if company is True:
            print(merged_data.groupby('company_id').applied.agg(['mean', 'count']))
        return merged_data.groupby('job_post_id').applied.agg(['mean', 'count'])
    elif company is True:
        return merged_data.groupby('company_id').applied.agg(['mean', 'count'])
    else:
        return 'ERROR: Neither company nor job was selected'

## Distance and time from postcodes

In [ ]:
def miles_to_metres(miles):
    return miles * MILES_TO_METRES


MILES_TO_METRES = 1.609344
inner_radius = {'East Midlands':  miles_to_metres(12), 'East of England': miles_to_metres(20),
                'London':  miles_to_metres(8), 'South East': miles_to_metres(12), 'South West': miles_to_metres(8),
                'West Midlands': miles_to_metres(8), 'Other': miles_to_metres(18)}
outer_radius = {'East Midlands':  miles_to_metres(40), 'East of England': miles_to_metres(25),
                'London':  miles_to_metres(15), 'South East':  miles_to_metres(29), 'South West':  miles_to_metres(20),
                'West Midlands': miles_to_metres(17), 'Other': miles_to_metres(24)}


def within_commute(distance, region='Other', commute_time=60):
    if commute_time != commute_time:
        commute_time = 60  # Basic distance to time conversion, based on regions
    if region not in inner_radius:
        region = 'Other'
    radii = inner_radius
    if commute_time < 40:
        flexibility = 1.25
    elif commute_time < 60:
        flexibility = 1.5
    elif commute_time < 90:
        flexibility = 2
    else:
        radii = outer_radius
        flexibility = 1
    within = (radii.get(region) * flexibility) - distance
    if within < 0:
        return 0
    else:
        return 1


def grouper(n, iterable, fillvalue=None):
    "grouper(3, 'ABCDEFG', 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(fillvalue=fillvalue, *args)

## Required for main script

In [ ]:
def preferrered_role_match(dataFrame):
    if dataFrame['preferred_role_types'] is None:
        return None
    preferred = dataFrame['preferred_role_types']
    job_post_id = dataFrame['job_post_id']
    jj = JobTalentPool[JobTalentPool['job_id'] == job_post_id]
    intersect = len(set(jj['_role_type_id']).intersection(set(preferred)))
    return intersect


def intersect_(dataFrame):
    talent_pools_j = JobTalentPool[JobTalentPool['job_id'] == dataFrame.job_post_id]
    talent_pools_c = CandidatesTalentPool[CandidatesTalentPool['candidate_id'] == dataFrame.candidate_id]
    intersect = len(set(talent_pools_j['_role_type_id']).intersection(set(talent_pools_c['_role_type_id'])))
    return intersect


def intersect_seniority_(dataFrame):  # Checks the intersection length talent pools factoring in seniority
    talent_pools_j = JobTalentPool[JobTalentPool['job_id'] == dataFrame.job_post_id]
    talent_pools_c = CandidatesTalentPool[CandidatesTalentPool['candidate_id'] == dataFrame.candidate_id]
    intersect_seniority = 0
    for talent in set(talent_pools_j['_role_type_id']).intersection(set(talent_pools_c['_role_type_id'])):
        j_sen = list(talent_pools_j[talent_pools_j['_role_type_id'] == talent]['seniority'])
        c_sen = list(talent_pools_c[talent_pools_c['_role_type_id'] == talent]['seniority'])[0]
        if c_sen in j_sen:
            intersect_seniority += 1
        else:
            c_qual = list(cc[cc['_role_type_id'] == talent]['seniority_qualifier'])[0]
            if c_qual == 'certain':
                continue
            elif len(j_sen) >= 1:
                intersect_seniority += 1
            elif c_sen == 'junior' or j_sen == 'senior':
                intersect_seniority += 1
    return intersect_seniority


def basically_tinder(values):  # Similar to intersect, but on an individual roletype basis
    talent_pools_j = JobTalentPool[JobTalentPool.job_id == values.job_post_id]['_role_type_id']
    talent_pools_c = CandidatesTalentPool[CandidatesTalentPool.candidate_id == values.candidate_id]['_role_type_id']
    intersect = set(talent_pools_j).intersection(set(talent_pools_c))
    for match_num in intersect:
        values["match_" + str(match_num)] = 1
    return values


def yearly_salary_to_unit(Job_Post):  # converts hourly and daily salaries to yearly
    salary = Job_Post['maximum_salary']
    unit = Job_Post['salary_unit']
    if unit == 'annually':
        return salary
    if unit == 'daily':
        return (salary * 260)
    if unit == 'hourly':
        return (salary * 2080)


def contract_employment(value):  # Do contract types match
    if int(value.contract_work == 1) ^ int(value.permanent_work == 1):
        if value.permanent_work == 1 and value.contract_type == 'temp':
            value.contract_no_match = 1
        if value.contract_work == 1 and value.contract_type == 'permanent':
            value.contract_no_match = 1
    if int(value.full_time_work == 1) ^ int(value.part_time_work == 1):
        if value.employment_type is None:
            value.employment_type = 'full-time'
        if value.full_time_work == 1 and value.employment_type == 'part-time':
            value.employment_no_match = 1
        if value.part_time_work == 1 and value.employment_type == 'full-time':
            value.employment_no_match = 1
    return value


def within_salary(values):  # Are Salaries compatible
    if values.minimum_wage >= values.maximum_salary:
        values.within_income = 1
    return values


def descriptor_max(value):  # Which is newest experience
    values = value[1].dropna(subset=['description'])
    if len(values) < 1:
        return None,None
    if all([values.start_year.isna().all(), values.end_year.isna().all()]):
        return values.iloc[0].description, value[0]
    elif any([values.start_year.max() >= values.end_year.max(), values.end_year.isna().all()]):
        return values.loc[values.start_year.idxmax(), 'description'], value[0]
    else:
        return values.loc[values.end_year.idxmax(), 'description'], value[0]


def descriptor_top3(value):  # Which is newest 3 experiences
    values = value[1].copy()
    for iter in range(3):
        if all([values.start_year.isna().all(), values.end_year.isna().all()]):
            return values[3-iter:].index.to_list()
        elif any([values.start_year.max() >= values.end_year.max(), values.end_year.isna().all()]):
            inds = values.start_year.idxmax()
            values.drop([inds], inplace=True)
        else:
            values.drop([values.end_year.idxmax()], inplace=True)
    return values.index.to_list()


def True_on_going(values, True_ongoing):  # Is last job ongoing
    if values.candidate_id in True_ongoing:
        values.is_last_job_ongoing = 1
        return values
    else:
        return values


def index_unable(value):  # changes a few things
    if len(value) == 1:
        return value
    value['seq_no'] += range(len(value))  # how many emails came before plus one
    value['been_sent'] = 1  # This is not the first email
    value.head(1)['been_sent'] = 0  # This is the first
    iteration = 1
    for ind in value.index[1:]:
        value.loc[ind, 'invites_applied'] = value.iloc[:iteration].applied.sum()
        if value['past_shortlist']:
            if value.iloc[:iteration].shortlisted.sum() > 0:
                value.loc[ind, 'past_shortlist'] = 1
        value.loc[ind, 'mean_of_invites_applied'] = value.head(value['seq_no'][ind]-1)['applied'].mean()
        #  mean of past application rate
        best_estimate = 0
        ind_pre = value.index[iteration-1:iteration][0]
        if value.loc[ind_pre, 'applied'] == 0:
            value.loc[ind, 'dry_spell'] = 1 + value.loc[ind_pre, 'dry_spell']
        else:  # Dry spell, how many invites since last application
            value.loc[ind, 'dry_spell'] = 0
        # Looks at similarity of past invites and whether they were applied to
        for ind2 in value.index[:iteration]:
            applied_ = -1
            estimate = confidential_salary = 0
            if value.loc[ind2, 'applied'] == 1: 
                applied_ = 1
            if value.loc[ind2, 'employment_type'] == value.loc[ind, 'employment_type']:
                estimate += 1
                value.loc[ind, 'similarity_emp'] += applied_
            if value.loc[ind2, 'contract_type'] == value.loc[ind, 'contract_type']:
                estimate += 0.6
                value.loc[ind, 'similarity_con'] += applied_
            if value.loc[ind2, 'company_id'] == value.loc[ind, 'company_id']:
                estimate += 0.6
                value.loc[ind, 'similarity_com'] += applied_
            if value.loc[ind2, 'role_id'] == value.loc[ind, 'role_id']:
                estimate += 1
                value.loc[ind, 'similarity_role'] += applied_
            if value.loc[ind2, 'job_title'] == value.loc[ind, 'job_title']:
                estimate += 1
                value.loc[ind, 'similarity_title'] += applied_
            if value.loc[ind2, 'sector_id'] == value.loc[ind, 'sector_id']:
                estimate += 0.6
                value.loc[ind, 'similarity_sect'] += applied_
            if value.loc[ind2, 'post_seniority_level'] == value.loc[ind, 'post_seniority_level']:
                estimate += 1
                value.loc[ind, 'similarity_level'] += applied_
            if value.loc[ind2, 'asap_start'] == value.loc[ind, 'asap_start']:
                estimate += 0.2
                value.loc[ind, 'similarity_asap'] += applied_
            if value.loc[ind2, 'confidential_salary'] != value.loc[ind, 'confidential_salary']:
                confidential_salary = 1
            elif value.loc[ind2, 'confidential_salary'] == value.loc[ind, 'confidential_salary']:
                estimate += 1
                value.loc[ind, 'similarity_confid'] += applied_
            elif value.loc[ind, 'maximum_salary'] >= (value.loc[ind2, 'maximum_salary'] - 3000):
                estimate += 1
                value.loc[ind, 'similarity_sal'] += applied_
            if estimate != 0:
                estimate = estimate/(7-confidential_salary) * -1
                estimate = estimate * (1 - value.loc[ind2, 'applied'] * 2)
            if abs(estimate) > abs(best_estimate):
                best_estimate = estimate
        value.loc[ind, 'similarity_applied'] = best_estimate
        iteration += 1
    return value

## Similarities between experience to job description

In [ ]:
def low_power_similarity(learning_data):
    descriptions = pd.DataFrame(columns=['candidate_id', 'description'])
    for a in CandidateEs.groupby(['candidate_id']):
        desc,cand_key = descriptor_max(a)
        descriptions = descriptions.append({'candidate_id' : cand_key , 'description' : desc}, ignore_index=True)

    descriptions.dropna(inplace=True)
    descriptions = descriptions.set_index('candidate_id')

    descriptions2 = pd.merge(JobProfile[['job_profile_id','candidate_id']], descriptions, how='inner', left_on='candidate_id',right_on='candidate_id')
    descriptions2 = pd.merge(learning_data[['job_profile_id','applied_x','seq_no']], descriptions2, how='inner', left_on='job_profile_id',right_on='job_profile_id')
    descriptions3 = descriptions2[descriptions2.seq_no==1]
    descriptions4 = descriptions3[descriptions3.description.str.len()>8]


    embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder/2")
    JobPosts = tuple(JobProfile.job_post_id.unique())
    Job_Post = pd.read_sql_query('''SELECT id, description FROM customer_jobpost WHERE id IN %s''' % (str(JobPosts)), cnx)
    Job_Post = Job_Post.rename(columns={'id':'job_post_id'})
    use_on_use = list(descriptions2.description)
    use_on_use2 = list(Job_Post.description)

    tfi = TfidfVectorizer(ngram_range=(1, 4), stop_words='english')
    X = tfi.fit_transform(descriptions4.description)

    temp = np.zeros(shape=(len(descriptions2.description),512))
    temp2 = np.zeros(shape=(X.shape[0],512))
    tf.logging.set_verbosity(tf.logging.ERROR)
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(use_on_use2))
        for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
            temp2[i,:] = message_embedding
    with tf.Session() as session2:
        session2.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session2.run(embed(use_on_use))
        for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
            temp[i,:] = message_embedding
    temp_=temp.copy()
    temp2_=temp2.copy()
    descriptions5 = pd.merge(descriptions2,JobProfile[['job_profile_id','job_post_id']], how='inner', left_on='job_profile_id',right_on='job_profile_id')
    temp_3 = np.zeros(temp_.shape)

    for i in range(len(descriptions5)):
        idx = Job_Post[Job_Post.job_post_id==495].index[0]
        temp3_[i] = temp2_[idx]
    score = np.clip(np.sum(np.multiply(temp_3, temp_), axis=1), a_min=-1.0,a_max=1.0)
    sim_score = 1 - np.arccos(score)
    descriptions5["U_S_E"] = sim_score #Used instead of session run_sts_benchmark
    descriptions5["U_S_E_ab"] = abs(descriptions5["U_S_E"])
    
    learning_data = pd.merge(learning_data, descriptions5[['job_profile_id','U_S_E']], how='inner', left_on='job_profile_id',right_on='job_profile_id')
    return learning_data

## Bag of words: experience position vs job title

In [ ]:
def intersection(lst1, lst2):
    temp = set(lst2) 
    lst3 = [value for value in lst1 if value in temp] 
    return len(lst3)


def title_intersection(learning_data, CandidateEs):
    JobProfile_limited = JobProfile[JobProfile.job_profile_id.isin(learning_data.job_profile_id.to_list())]
    Job_Post_limited = Job_Post[Job_Post.job_post_id.isin(JobProfile_limited.job_post_id.to_list())]
    Job_Post_limited = Job_Post[Job_Post.job_post_id.isin(JobProfile_limited.job_post_id.to_list())]
  
    list_keep=[]
    RecentCandidateEs = CandidateEs.dropna(subset=['description'])
    descriptions = RecentCandidateEs[['candidate_id','position','description', 'experience_id']]
    for a in RecentCandidateEs.groupby(['candidate_id']):
        if len(a[1]) > 3:
            ind = descriptor_top3(a)
            list_keep.extend(ind)

    descriptions.drop(list_keep, inplace=True)

    descriptions2 = descriptions.copy()
    descriptions2.dropna(subset = ['position'], inplace=True)
    descriptions2.position = descriptions2.position.str.lower()
    descriptions2.position = descriptions2.position.str.findall(r'[^\s!,./()-?&%":;0-9]+')
    descriptions2.position = [w for w in descriptions2.position if not w in ['and', 'to', 'this', 'the']]

    post_to_cv = pd.merge(descriptions2, JobProfile, how="inner", left_on="candidate_id", right_on="candidate_id")
    post_to_cv = pd.merge(post_to_cv, Job_Post, how="inner", left_on="job_post_id", right_on="job_post_id")
    post_to_cv["title_intersect"] = post_to_cv["descript_intersect"] = 0

    post_to_cv.dropna(subset = ['job_title'], inplace=True)
    post_to_cv.job_title = post_to_cv.job_title.str.lower()
    post_to_cv.job_title = post_to_cv.job_title.str.findall(r'[^\s!,./()-?&%":;0-9]+')
    post_to_cv.job_title = [w for w in post_to_cv.job_title if not w in ['and', 'to', 'this', 'the']]
    
    ddata = dd.from_pandas(post_to_cv, npartitions=50)
    post_to_cv['title_intersect'] = ddata.map_partitions(lambda df: df.apply((lambda row: intersection(row.position, row.job_title)), axis=1)).compute(scheduler='processes')
    
    jps_dic = post_to_cv[['job_profile_id','title_intersect']].set_index('job_profile_id').to_dict()['title_intersect']
    learning_data['title_intersect'] = 0
    idx = learning_data[learning_data.job_profile_id.isin(post_to_cv.job_profile_id.to_list())].index
    learning_data.loc[idx, 'title_intersect'] = [jps_dic[w] for w in learning_data.loc[idx, 'job_profile_id']]
    
    return learning_data

## Main script

In [ ]:
def load_data_to_local():
    Job_Post = pd.read_sql_query('''SELECT id,does_not_require_industry_experience,
    does_not_require_practice_area_experience,sector_id,employment_type,job_title,role_id,
    weekly_hours,asap_start,holidays,level,company_id,location_info,
    maximum_salary,salary_unit,does_not_require_sector_experience,contract_type,
    confidential_salary,postcode FROM customer_jobpost;''', cnx)
    JobProfile = pd.read_sql_query('''SELECT id,candidate_id,job_post_id FROM  customer_JobProfile;''', cnx)
    Companies = pd.read_sql_query('''SELECT id,size,name FROM company_company;''', cnx)
    Candidates = pd.read_sql_query('''SELECT id,permanent_work,temp_to_perm_work,contract_work,
    full_time_work,part_time_work,minimum_wage,postcode,commute_time,commute_by_car  FROM candidate_candidate;''', cnx)
    CandidatesTalentPool = pd.read_sql_query('''SELECT candidate_id,seniority_qualifier,
    _role_type_id,seniority FROM ops_CandidateTalentPool;''', cnx)
    JobTalentPool = pd.read_sql_query('''SELECT job_id,_role_type_id,seniority FROM ops_JobTalentPool;''', cnx)

    Job_Post = Job_Post.rename(columns={'id': 'job_post_id', 'does_not_require_sector_experience': 'req_sector_exp',
                                        'does_not_require_practice_area_experience': 'req_practice_exp',
                                        'does_not_require_industry_experience': 'req_industry_exp'})
    JobProfile = JobProfile.rename(columns={'id': 'job_profile_id'})
    Companies = Companies.rename(columns={'id': 'company_id', 'size': 'size_company'})
    Candidates = Candidates.rename(columns={'id': 'candidate_id', 'postcode': 'postcode_c',
                                            '_preferred_role_types': 'preferred_role_types'})
    Job_Post['maximum_salary'] = Job_Post.apply(yearly_salary_to_unit, axis=1)
    Job_Post = Job_Post.drop(['salary_unit'], axis=1)
    return Job_Post, JobProfile, Companies, Candidates, CandidatesTalentPool, JobTalentPool


def information_for_regression(data):
    data["week_day"] = [w.weekday() for w in data.created]
    data["hour"] = [w.hour for w in data.created]
    data.created = data.created.dt.year * 12 + data.created.dt.month
    data.created = data.created - min(data.created)
    
    data_collation = pd.merge(data, JobProfile, how="inner", left_on="job_profile_id",
                              right_on="job_profile_id").sort_values(['created'])
    data_collation['seq_no'] = 1
    data_collation['been_sent'] = data_collation['mean_of_invites_applied'] = data_collation['past_shortlist'], data_collation['invites_applied']ource = 0
    for col in ['emp', 'con', 'com', 'role', 'title', 'sect', 'level', 'asap', 'confid', 'sal', 'applied']:
        data_collation["similarity_" + col] = 0
    data_collation = pd.merge(data_collation, Job_Post, how="inner", left_on="job_post_id", right_on="job_post_id")
    data_collation = data_collation.groupby(['candidate_id']).apply(index_unable)
    data_collation = pd.merge(data_collation, Candidates, how="inner", left_on="candidate_id", right_on="candidate_id")
    data_collation = pd.merge(data_collation, Companies, how="inner", left_on="company_id", right_on="company_id")
    data_collation = data_collation.dropna(subset=['minimum_wage'])
    data_ = get_sourcing()  # Where each candidate was sourced from
    data_.job_profile_id = pd.to_numeric(data_.job_profile_id, errors='coerce')
    data_collation = pd.merge(data_collation, data_, how="inner", left_on="job_profile_id", right_on="job_profile_id")
    data_collation.set_index(pd.Index(range(len(data_collation))), inplace=True)

    df_c = pd.concat([data_collation["postcode_c"], data_collation["postcode"]], axis=1)  # List of unique postcodes
    for a in df_c:
        a = a.lower()

    results = []
    unique_postcodes = []
    for a in range(len(df_c)):
        pc1 = df_c["postcode_c"][a]
        pc2 = df_c["postcode"][a]
        unique_postcodes.append(pc1)
        unique_postcodes.append(pc2)
        unique_postcodes = list(set(unique_postcodes))

    for pcs in tqdm(list(grouper(90, unique_postcodes))):  # range((unique_postcodes.shape[0] // 100)+1)):
        st = {"postcodes": pcs}
        r = requests.post(url='http://api.postcodes.io/postcodes', data=st)  # converts to long and lat
        results.extend(r.json()['result'])

    postcodes_info = pd.DataFrame(results)
    postcodes_info = postcodes_info.apply(lambda x: pd.Series(x['result']), axis=1)
    postcodes_info = postcodes_info.dropna(subset=['postcode']).drop_duplicates(subset=['postcode'])
    postcodes_info = postcodes_info.set_index(postcodes_info['postcode'].str.replace(' ', '').str.lower())

    data_collation["distance"] = df_c.apply(lambda y: distance_between_postcodes(y['postcode_c'], y['postcode'],
                                                                                 postcodes_info), axis=1)
    data_collation['region'] = data_collation.location_info.apply(lambda x: x.get("region"))
    data_collation.dropna(subset=['distance'], inplace=True)
    data_collation['within_distance'] = data_collation.apply(lambda x: within_commute(x['distance'], x['region'],
                                                                                      x['commute_time']), axis=1)
    data_collation['intersect_seniority'] = data_collation.apply(intersect_seniority_, axis=1)
    data_collation['intersect'] = data_collation.apply(intersect_, axis=1)

    data_collation['contract_no_match'] = data_collation['employment_no_match'] = 0
    data_collation = data_collation.apply(contract_employment, axis=1)
    data_collation = data_collation[((np.abs(stats.zscore(data_collation['minimum_wage']))) < 3)]  # removes oddities

    learning_data = data_collation.copy()
    '''[['level','applied','candidate_id', 'job_post_id', 'intersect', 'intersect_seniority', 'seq_no', 'sector_id',
    'mean_of_invites_applied', 'been_sent', 'source_i_1', 'source_i_2', 'source_i_3', "job_profile_id",
    'source_e_1', 'source_e_2', 'source_e_3', 'source_e_4', 'source_e_5', 'source_e_6', 'source_e_7', 'source_e_8'
    'source_e_10', 'source_e_11', 'distance', 'minimum_wage', 'contract_no_match', 'asap_start', 'size_company',
    'holidays', 'employment_no_match', 'maximum_salary', 'confidential_salary', 'within_distance', 'source_e_9',
    'req_industry_exp', 'req_sector_exp', "similarity_applied", 'req_practice_exp']]'''
    learning_data.dropna(subset=['size_company', 'distance'], inplace=True)
    learning_data.holidays.fillna(learning_data.holidays.mean(), inplace=True)
    learning_data = learning_data.rename(columns={'level': 'post_seniority_level'})

    Job_roles = list(JobTalentPool._role_type_id.value_counts().index)
    Cand_roles = list(CandidatesTalentPool._role_type_id.value_counts().index)
    role_matches = list(set(Job_roles) & set(Cand_roles))
    columns_match = []
    for role_type in role_matches:
        learning_data["match_" + str(role_type)] = 0
        columns_match.append("match_" + str(role_type))

    learning_data = learning_data.apply(basically_tinder, axis=1)
    for col in columns_match:
        if learning_data[col].mean() < 0.001:
            learning_data = learning_data.drop([col], axis=1)

    learning_data = pd.get_dummies(learning_data, prefix=['size', 'sector'], columns=['size_company', 'sector_id'])

    learning_data['confidential_salary'] = learning_data['confidential_salary'].astype(int)
    learning_data['asap_start'] = learning_data['asap_start'].astype(int)
    learning_data['req_sector_exp'] = learning_data['req_sector_exp'].astype(int)
    learning_data['req_industry_exp'] = learning_data['req_industry_exp'].astype(int)
    learning_data['req_practice_exp'] = learning_data['req_practice_exp'].astype(int)
    learning_data = learning_data.set_index(pd.Index(range(len(learning_data))))

    learning_data['within_income'] = 0
    learning_data = learning_data.apply(within_salary, axis=1)
    learning_data['minimum_wage'] = np.log(learning_data['minimum_wage'])
    learning_data['maximum_salary'] = np.log(learning_data['maximum_salary'])

    CandidateEs = pd.read_sql_query('''SELECT * FROM candidate_CandidateExperience;''', cnx)
    cand_ids = learning_data.candidate_id.unique()
    CandidateEs = CandidateEs[CandidateEs.candidate_id.isin(cand_ids)]
    True_ongoing = CandidateEs[CandidateEs.ongoing is True].candidate_id.unique()
    learning_data['is_last_job_ongoing'] = 0
    cand_ids = learning_data[learning_data.candidate_id.isin(True_ongoing)].index
    learning_data.loc[cand_ids, 'is_last_job_ongoing'] = 1
    # learning_data = learning_data.apply(lambda x: True_on_going(x,True_ongoing), axis = 1) |replaces previous 2 lines

    last_updated = CandidateEs.groupby('candidate_id')[['start_year', 'end_year']].max()
    last_updated['years_since_cv_update'] = 2019 - last_updated[['start_year', 'end_year']].max(axis=1)
    last_updated = last_updated.drop(['start_year', 'end_year'], axis=1)
    last_updated.years_since_cv_update = last_updated.years_since_cv_update.fillna(2)  # CandidateEs.duration.dt.days
    learning_data = pd.merge(learning_data, last_updated, how='outer', left_on='candidate_id', right_index=True)
    learning_data = learning_data.sort_index()
    learning_data.years_since_cv_update = learning_data.years_since_cv_update.fillna(2)  # arb

    learning_data['sim_not_applied'] = learning_data['similarity_applied']  # splitting negative and positive
    learning_data.sim_not_applied = learning_data.sim_not_applied.where(learning_data.sim_not_applied < 0, 0)
    learning_data.similarity_applied = learning_data.similarity_applied.where(learning_data.similarity_applied > 0, 0)
    learning_data['similarity_applied_sq'] = learning_data['similarity_applied']**2
    learning_data['sim_not_applied_sq'] = learning_data['sim_not_applied']**2

    learning_data = learning_data.drop(['candidate_id', 'job_post_id'], axis=1)
    
    learning_data = title_intersection(learning_data)  # similarity between last experience and job post description
    learning_data = title_intersection(learning_data)  # Bag of words: experience position vs job title
    
    return learning_data

## Calling

In [ ]:
data = import_data(cnx)
Job_Post, JobProfile, Companies, Candidates, CandidatesTalentPool, JobTalentPool = load_data_to_local()
regression_data = information_for_regression(data)
# regression_data.to_excel("regression_data.xlsx")

## Modeling with data

In [ ]:
# %%pycodestyle
X_train, X_test, y_train, y_test = train_test_split(X, regression_data.applied, test_size=0.2)
model = xgb.XGBClassifier(max_depth=10, scale_pos_weight=3.4)
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))